# FFF Vision - Canny Edge Detection Method

In [5]:
#Importing all necessary packages.
from cv2 import pointPolygonTest
import numpy as np
import cv2
print('OpenCV version: '+cv2.__version__)
import pandas as pd
import datetime
import os
from collections import Counter
import time

OpenCV version: 4.6.0


In [6]:
#used for time calculation.
start = time.perf_counter()
#Reading the folder where the videos are saved.
SRC_FOLDER = r"D:\Pensulvaniya state project\FFF Vision - OpenCV [External]\Vidseos\\"
#reading the vieo_Time_Stamp from device.
df_vidTimes = pd.read_excel(SRC_FOLDER + "Video_Timestamps_1.xlsx")
df_vidTimes.drop(df_vidTimes.columns[0],axis=1,inplace=True)

Image Processing Steps:

Perspective Correction -> Binary Thershold -> Morphological Operations -> Median Blur Operations -> Canny Edge detection ->ROI Selection

In [7]:
#Function for image_perspective correction.
def perspCorrection(img,pt1,pt2,pt3,pt4,scale_width,scale_height):
    
    
    input_pts = np.float32([pt1,pt2,pt3,pt4])
    output_pts = np.float32([[0,0],[scale_width-1,0],[0,scale_height-1],[scale_width-1,scale_height-1]])
    M = cv2.getPerspectiveTransform(input_pts,output_pts)
    imgPersp = cv2.warpPerspective(img,M,(scale_width, scale_height))
    #grayscale conversion of the perspective corrected image
    imgGrayPersp = cv2.cvtColor(imgPersp, cv2.COLOR_BGR2GRAY)            
    
    return [imgPersp,imgGrayPersp]


In [8]:
#function for binary thersholding
def thersold_fun(thersholded):
    thersholded = cv2.threshold(imgGrayPersp, 127, 255, cv2.THRESH_BINARY)[1]
    return thersholded

In [9]:
#function for morphological operations(dilation & erosion).
def morphOps(medianblr):    

    mask_erosion = cv2.erode(medianblr, np.ones((8, 0), "uint8"))
    mask_dilation = cv2.dilate(mask_erosion, np.ones((32, 0), "uint8"))
    return mask_dilation

In [10]:
#function for median bluring.
def medianBlur(imgGrayPersp):

    mblur = cv2.medianBlur(imgGrayPersp, 9)
    return mblur

In [11]:
#function for canny edge detection
def canny(cloperation,threshold_1, threshold_2):

    canny_image = cv2.Canny(cloperation,threshold_1, threshold_2) 
    return canny_image

In [12]:
#extracting ROI from edge detected image.
def extractTopBottom(img,bStart,bEnd):
   
    img_bottom = img[bStart[1]:bEnd[1],bStart[0]:bEnd[0]]      
    
    return img_bottom

In [13]:
#function for computing extrusion width from detected edges.
def computeW(canny,mm_per_pixel,imgGrayPersp):
    indices = np.where(canny != [0])
    LEdge = np.mean(indices[1][::2])
    REdge = np.mean(indices[1][1::2])
    # lefavg = np.mean(LEdge)
    # rigavg = np.mean(REdge)
    widths = np.abs(REdge-LEdge)
    #width = np.mean(widths)
    width_in_mm = mm_per_pixel * widths
    # width1 = np.abs(rigavg-lefavg)
    # width_in_mm1 = mm_per_pixel * width1
    text = str(width_in_mm) + ' mm'
    font = cv2.FONT_HERSHEY_SIMPLEX
    org = (50,50)
    fontScale = 1
    color = (255, 0, 0)
    thickness = 2
    img_color = cv2.putText(imgGrayPersp, text, org, font,fontScale, color, thickness)
         
    return [widths, width_in_mm,img_color]

In [14]:
#function for storing images for analysis, need to call this function from main function whenever needed.
#This function will save the output of all indivigual functions into seperated folder for debugging.
#There should be image folders as mentioned in the save_path
def image_for_analysis(SRC_FOLDER,layers,speeds,imgGrayPersp,thersholded_image,medianblr,cloperation,canny_image,bottom_image,frameCount,imgcolor):
    
    #for storing perspective corrected grayscale images.
    pCorrImg_savePath = SRC_FOLDER + "Results/Run-2/" + str(layers) + "/" + str(speeds) + "/Gray/"
    cv2.imwrite(pCorrImg_savePath + "pCorr" + str(frameCount) + ".jpg", imgGrayPersp)
    #for storing thersholded images.                       
    threshImg_savePath = SRC_FOLDER + "Results/Run-2/"+ str(layers) + "/" + str(speeds) + "/Thersholded Images/"
    cv2.imwrite(threshImg_savePath + "thresh" + str(frameCount) + ".jpg", thersholded_image)
    #for storing output images of  median blur function.                     
    binImg_savePath = SRC_FOLDER + "Results/Run-2/" + str(layers) + "/" + str(speeds) + "/median Blur/"
    cv2.imwrite(binImg_savePath + "medianblur" + str(frameCount) + ".jpg", medianblr)
    #for storing output images of  morphological operations(dilation and erosion).                     
    cloperation_savePath = SRC_FOLDER + "Results/Run-2/" + str(layers) + "/" + str(speeds) + "/Erosion and Dilation/"
    cv2.imwrite(cloperation_savePath + "morphOps" + str(frameCount) + ".jpg", cloperation)
    #for storing canny images with detected edges.                         
    canny_image_savePath = SRC_FOLDER + "Results/Run-2/"+ str(layers) + "/" + str(speeds) + "/Canny/"
    cv2.imwrite(canny_image_savePath + "Canny" + str(frameCount) + ".jpg", canny_image)
    #for storing the ROI selected images.                        
    bottom_image_savePath = SRC_FOLDER +"Results/Run-2/"+ str(layers) + "/" + str(speeds) + "/ROI/"
    cv2.imwrite(bottom_image_savePath + "ROI" + str(frameCount) + ".jpg", bottom_image)    
    #for storing the images with labeled width measures.
    wImg_savePath = SRC_FOLDER +"Results/Run-2/"+ str(layers) + "/" + str(speeds) + "/Vision Measurements/"
    cv2.imwrite(wImg_savePath + "wImg" + str(frameCount) +str(layers)  + str(speeds)+ ".jpg", imgcolor)
    
    return None
#This function returns nothing since it only save images to respective folders in system.


In [16]:
#Main program
num_layers = 20
max_speed = 50
layers = list(range(5,num_layers+1))
speeds = list(range(10,max_speed+10,10))
frame_skip_start = [32,20,11,15,13] 

vidCount = 0
img_debug = False 

w_result_columns=['Layer','vR','Frame','ActualTimestamp','Edge Distance in Pixel','w_Vision']
frame_summary_columns = ['Layer','vR','Start_TS','End_TS','Total_Frames','Per_Frames_Skipped','Skipped_Frames']
lst = []
lst_skip_frames = []
lst_frame_summary = []

pt1 = [192.30,343.00]  
pt2 = [1079.0,379.80]  
pt3 = [153.50,571.90] 
pt4 = [1107.10,611.70] 

scale_width = round(11.7348*200) 
scale_height = round(6.35*200)   

bStart = [655,925]
bEnd = [1300,1270]
fsize =9
threshold_1 =30
threshold_2 = 80
mm_per_pixel = 0.004992138364779874

for l in range(len(layers)):
    for v in range(len(speeds)):
        
        lst = []
        lst_skip_frames = []
                
        vidName = 'vid_l'+str(layers[l])+'_vR_'+str(speeds[v])+'.avi'
        print('Processing video: ' + vidName)
        
        idx = df_vidTimes.index[(df_vidTimes.Layer==layers[l]) & (df_vidTimes.Speed==speeds[v])].to_list()[0]
        start_TS = df_vidTimes.Start_Timestamp[idx]
        end_TS = df_vidTimes.End_Timestamp[idx]
        
        print('video: {0} starts at {1} and ends at {2}'.format(vidName,start_TS,end_TS))
        
        srcVideo = SRC_FOLDER + vidName       
        cap = cv2.VideoCapture(srcVideo)
        numFrames = cap.get(cv2.CAP_PROP_FRAME_COUNT)
        print('video {0} has {1} frames'.format(vidName,numFrames))

        if (cap.isOpened() == False):
            print("Error reading video file. Exiting ...")
            exit(0)
     
            
            
        frameCount = 0

        while(cap.isOpened()):
    
            frame_exists, frame = cap.read()
               
               
            if frame_exists:
                
                frameCount = frameCount + 1
                

                if(frameCount >= frame_skip_start[v] and frameCount <= numFrames - 5): 
                    
                    try:
                                               

                        # call function - correct perspective transform
                        [imgPersp,imgGrayPersp] = perspCorrection(frame,pt1,pt2,pt3,pt4,scale_width,scale_height)
                        #thersholding image using thresh.binary function
                        thersholded_image = thersold_fun(imgGrayPersp)
                        #closing operation
                        cloperation = morphOps(thersholded_image)
                        #medianblur
                        medianblr = medianBlur(cloperation)
                        #Canny detection
                        canny_image= canny(medianblr,threshold_1, threshold_2)
                        #ROI selection
                        bottom_image = extractTopBottom(canny_image,bStart,bEnd)
                        # Extrusion width measurement
                        [bottom_edge_pixels,bottom_edge_dist,img_color] = computeW(bottom_image,mm_per_pixel,imgGrayPersp) 
                        
                        #Remove Comment line for below code , if images needs to be saved for analysing.
                        #image_for_analysis(SRC_FOLDER,str(layers[l]),str(speeds[v]),imgGrayPersp,thersholded_image,medianblr,cloperation,canny_image,bottom_image,frameCount,imgcolor)

                        # Calculate actual timestamp based on excel timestamps and frame number
                        act_TS = start_TS+frameCount*(end_TS-start_TS)/numFrames

                        # Store results in dataframe   
                        lst.append([layers[l],speeds[v],frameCount,act_TS,bottom_edge_pixels,bottom_edge_dist])
                        

                        
                    except ValueError as e:
                        
                        print('Unable to sucessfully process frame {0}, skipping . . .'.format(frameCount))
                        print(e)
                        # Calculate actual timestamp based on excel timestamps and frame number
                        act_TS = start_TS+frameCount*(end_TS-start_TS)/numFrames
                        # Store results in dataframe   
                        lst.append([layers[l],speeds[v],frameCount,act_TS,np.nan,np.nan])
                        lst_skip_frames.append([frameCount])

                    except UnboundLocalError as u:
                        print('Unable to sucessfully process frame {0}, skipping . . .'.format(frameCount))
                        print(u)
                        # Calculate actual timestamp based on excel timestamps and frame number
                        act_TS = start_TS+frameCount*(end_TS-start_TS)/numFrames
                        # Store results in dataframe   
                        lst.append([layers[l],speeds[v],frameCount,act_TS,np.nan,np.nan])
                        lst_skip_frames.append([frameCount])

            else:
                
                break       
                   
        cap.release()
        cv2.destroyAllWindows()
        
        print('Finished processing video: {0}'.format(vidName))
        print('')
        print('')
        vidCount = vidCount + 1
    
        results = pd.DataFrame(lst,columns=w_result_columns)
        # Save results to excel
        path = SRC_FOLDER + "Results/Run-2/" + 'l' + str(layers[l])+'_vR'+str(speeds[v])+"results1.csv"
        results.to_csv(path)
        lst_frame_summary.append([layers[l],speeds[v],start_TS,end_TS,numFrames,len(lst_skip_frames)/numFrames,lst_skip_frames])


frame_summary_results = pd.DataFrame(lst_frame_summary,columns=frame_summary_columns)

# Some more cleanup and data addition
frame_summary_results["Video_Duration"] = frame_summary_results["End_TS"] - frame_summary_results["Start_TS"] 
frame_summary_results["Video_Duration"] = [x.total_seconds() for x in frame_summary_results["Video_Duration"]]
frame_summary_results["FPS"] = frame_summary_results["Total_Frames"]/frame_summary_results["Video_Duration"]
frame_summary_results["Total_Frames_Skipped"] = [len(x) for x in frame_summary_results["Skipped_Frames"]]
# Re-oder columns
frame_summary_results = frame_summary_results[["Layer","vR","Start_TS","End_TS","Video_Duration","Total_Frames","FPS","Total_Frames_Skipped","Per_Frames_Skipped","Skipped_Frames"]]
#Writing to CSV
path_summary = SRC_FOLDER + "Results/Run-2/"+ 'video_processing_summary.csv'
frame_summary_results.to_csv(path_summary)
    
print('Processing of all videos completed successfully! Summary results saved at {0}'.format(SRC_FOLDER + 'video_processing_summary.xlsx'))
end = time.perf_counter() - start
print('{:.6f}s for the calculation'.format(end))

Processing video: vid_l5_vR_10.avi
video: vid_l5_vR_10.avi starts at 2021-11-19 21:37:35.690000 and ends at 2021-11-19 21:37:55.947000
video vid_l5_vR_10.avi has 274.0 frames
Finished processing video: vid_l5_vR_10.avi


Processing video: vid_l5_vR_20.avi
video: vid_l5_vR_20.avi starts at 2021-11-19 21:38:00.103000 and ends at 2021-11-19 21:38:09.781000
video vid_l5_vR_20.avi has 129.0 frames
Finished processing video: vid_l5_vR_20.avi


Processing video: vid_l5_vR_30.avi
video: vid_l5_vR_30.avi starts at 2021-11-19 21:38:22.880000 and ends at 2021-11-19 21:38:30.198000
video vid_l5_vR_30.avi has 99.0 frames
Finished processing video: vid_l5_vR_30.avi


Processing video: vid_l5_vR_40.avi
video: vid_l5_vR_40.avi starts at 2021-11-19 21:38:34.408000 and ends at 2021-11-19 21:38:43.093000
video vid_l5_vR_40.avi has 116.0 frames
Finished processing video: vid_l5_vR_40.avi


Processing video: vid_l5_vR_50.avi
video: vid_l5_vR_50.avi starts at 2021-11-19 21:38:13.970000 and ends at 2021-11-1

Finished processing video: vid_l12_vR_30.avi


Processing video: vid_l12_vR_40.avi
video: vid_l12_vR_40.avi starts at 2021-11-19 21:46:53.761000 and ends at 2021-11-19 21:47:02.438000
video vid_l12_vR_40.avi has 113.0 frames
Finished processing video: vid_l12_vR_40.avi


Processing video: vid_l12_vR_50.avi
video: vid_l12_vR_50.avi starts at 2021-11-19 21:46:33.327000 and ends at 2021-11-19 21:46:38.065000
video vid_l12_vR_50.avi has 66.0 frames
Finished processing video: vid_l12_vR_50.avi


Processing video: vid_l13_vR_10.avi
video: vid_l13_vR_10.avi starts at 2021-11-19 21:47:06.373000 and ends at 2021-11-19 21:47:26.613000
video vid_l13_vR_10.avi has 285.0 frames
Finished processing video: vid_l13_vR_10.avi


Processing video: vid_l13_vR_20.avi
video: vid_l13_vR_20.avi starts at 2021-11-19 21:47:30.783000 and ends at 2021-11-19 21:47:40.472000
video vid_l13_vR_20.avi has 136.0 frames
Finished processing video: vid_l13_vR_20.avi


Processing video: vid_l13_vR_30.avi
video: vid_l13_vR_

Finished processing video: vid_l19_vR_50.avi


Processing video: vid_l20_vR_10.avi
video: vid_l20_vR_10.avi starts at 2021-11-19 21:55:25.896000 and ends at 2021-11-19 21:55:46.073000
video vid_l20_vR_10.avi has 278.0 frames
Finished processing video: vid_l20_vR_10.avi


Processing video: vid_l20_vR_20.avi
video: vid_l20_vR_20.avi starts at 2021-11-19 21:55:50.310000 and ends at 2021-11-19 21:55:59.984000
video vid_l20_vR_20.avi has 134.0 frames
Finished processing video: vid_l20_vR_20.avi


Processing video: vid_l20_vR_30.avi
video: vid_l20_vR_30.avi starts at 2021-11-19 21:56:13.110000 and ends at 2021-11-19 21:56:20.429000
video vid_l20_vR_30.avi has 98.0 frames
Finished processing video: vid_l20_vR_30.avi


Processing video: vid_l20_vR_40.avi
video: vid_l20_vR_40.avi starts at 2021-11-19 21:56:24.641000 and ends at 2021-11-19 21:56:33.396000
video vid_l20_vR_40.avi has 126.0 frames
Finished processing video: vid_l20_vR_40.avi


Processing video: vid_l20_vR_50.avi
video: vid_l20_vR_